# 简介
- [函数式编程](https://coolshell.cn/articles/10822.html)
- [Python修饰器的函数式编程](http://www.cnblogs.com/zhongguiyao/p/7208519.html)

In [6]:
def hello(fn):
    def wrapper():
        print("hello, %s" % fn.__name__)
        fn()
        print("goodby, %s" % fn.__name__)
    return wrapper


@hello
def foo():
    print("I am foo")


foo()

hello, foo
I am foo
goodby, foo


## Decorator 的本质
对于Python的这个`@`注解语法糖- Syntactic Sugar 来说，当你在用某个`@decorator`来修饰某个函数func时，如下所示:
```py
@decorator
def func():
    pass
```
其解释器会解释成下面这样的语句：
```py
func = decorator(func)
```

In [12]:
hello(foo())

hello, foo
I am foo
goodby, foo


<function __main__.hello.<locals>.wrapper>

In [13]:
def fuck(fn):
    print("fuck %s!" % fn.__name__[::-1].upper())


@fuck
def wfg():
    pass

fuck GFW!


### 多个 decorator
```py
@decorator_one
@decorator_two
def func():
    pass
```
相当于：
```py
func = decorator_one(decorator_two(func))
```
### 比如：带参数的 decorator：
```
@decorator(arg1, arg2)
def func():
    pass
```
相当于：
```
func = decorator(arg1,arg2)(func)
```

## 带参数及多个 Decrorator

In [7]:
def makeHtmlTag(tag, *args, **kwds):
    def real_decorator(fn):
        css_class = " class='{0}'".format(kwds["css_class"]) \
            if "css_class" in kwds else ""

        def wrapped(*args, **kwds):
            return "<"+tag+css_class+">" + fn(*args, **kwds) + "</"+tag+">"
        return wrapped
    return real_decorator


@makeHtmlTag(tag="b", css_class="bold_css")
@makeHtmlTag(tag="i", css_class="italic_css")
def hello():
    return "hello world"


print(hello())

<b class='bold_css'><i class='italic_css'>hello world</i></b>


## `class`式的 Decorator

In [8]:
class myDecorator(object):

    def __init__(self, fn):
        print("inside myDecorator.__init__()")
        self.fn = fn

    def __call__(self):
        self.fn()
        print("inside myDecorator.__call__()")


@myDecorator
def aFunction():
    print("inside aFunction()")


print("Finished decorating aFunction()")
aFunction()

inside myDecorator.__init__()
Finished decorating aFunction()
inside aFunction()
inside myDecorator.__call__()


## 用 Decorator 设置函数的调用参数

你有三种方法可以干这个事：
### 第一种: `**kwargs`
这种方法 decorator 会在 `kwargs` 中注入参数。

In [16]:
def decorate_A(function):
    def wrap_function(*args, **kwargs):
        kwargs['str'] = 'Hello!'
        return function(*args, **kwargs)
    return wrap_function


@decorate_A
def print_message_A(*args, **kwargs):
    print(kwargs['str'])


print_message_A()

Hello!


### 第二种，约定好参数，直接修改参数

In [17]:
def decorate_B(function):
    def wrap_function(*args, **kwargs):
        str_ = 'Hello!'
        return function(str_, *args, **kwargs)
    return wrap_function


@decorate_B
def print_message_B(str_, *args, **kwargs):
    print(str_)


print_message_B()

Hello!


### 第三种，通过 `*args` 注入

In [15]:
def decorate_C(function):
    def wrap_function(*args, **kwargs):
        str_ = 'Hello!'
        #args.insert(1, str)
        args = args + (str,)
        return function(*args, **kwargs)
    return wrap_function


class Printer:
    @decorate_C
    def print_message(self, str_, *args, **kwargs):
        print(str_)


p = Printer()
p.print_message()

Hello!


## Decorator的副作用
如果你查询一下`foo.__name__`的话，你会发现其输出的是 “wrapper”，而不是我们期望的 “foo”，这会给我们的程序埋一些坑。所以，Python 的`functool`包中提供了一个叫`wrap`的 decorator 来消除这样的副作用。下面是我们新版本的 `hello.py`。

In [25]:
from functools import wraps


def hello(fn):
    def wrapper():
        print("hello, %s" % fn.__name__)
        fn()
        print("goodby, %s" % fn.__name__)
    return wrapper


@hello
def foo():
    '''foo help doc'''
    print("I am foo")


foo()
print(30*'_')
print(foo.__name__)  # 输出 wrapper

print(30*'_')
print(foo.__doc__)  # 输出 None

hello, foo
I am foo
goodby, foo
______________________________
wrapper
______________________________
None


当然，即使是你用了`functools`的`wraps`，也不能完全消除这样的副作用。
看下面这个示例：

```py
from inspect import getmembers, getargspec
from functools import wraps


def wraps_decorator(f):
    @wraps(f)
    def wraps_wrapper(*args, **kwargs):
        return f(*args, **kwargs)
    return wraps_wrapper


class SomeClass(object):
    @wraps_decorator
    def method(self, x, y):
        pass


obj = SomeClass()
for name, func in getmembers(obj, predicate=inspect.ismethod):
    print("Member Name: %s" % name)
    print("Func Name: %s" % func.__name__)
    print("Args: %s" % getargspec(func)[0])

# 输出：
# Member Name: method
# Func Name: method
# Args: []
```
你会发现，即使是你你用了`functools`的`wraps`，你在用`getargspec`时，参数也不见了。
要修正这一问，我们还得用 Python 的反射来解决，下面是相关的代码：
```py
def get_true_argspec(method):
    argspec = inspect.getargspec(method)
    args = argspec[0]
    if args and args[0] == 'self':
        return argspec
    if hasattr(method, '__func__'):
        method = method.__func__
    if not hasattr(method, 'func_closure') or method.__closure__ is None:
        raise Exception("No closure for method.")

    method = method.__closure__[0].cell_contents
    return get_true_argspec(method)
```
当然，我相信大多数人的程序都不会去`getargspec`。所以，用`functools`的`wraps`应该够用了。

## 一些 decorator 的示例

### 给函数调用做缓存
这个例实在是太经典了，整个网上都用这个例子做 decorator 的经典范例。

In [27]:
from functools import wraps


def memo(fn):
    cache = {}
    miss = object()

    @wraps(fn)
    def wrapper(*args):
        result = cache.get(args, miss)
        if result is miss:
            result = fn(*args)
            cache[args] = result
        return result

    return wrapper


@memo
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)

上面这个例子中，是一个斐波拉契数例的递归算法。我们知道，这个递归是相当没有效率的，因为会重复调用。比如：我们要计算`fib(5)`，于是其分解成`fib(4) + fib(3)`，而`fib(4)`分解成`fib(3)+fib(2)`，`fib(3)`又分解成`fib(2)+fib(1)`…… 你可看到，基本上来说，`fib(3), fib(2), fib(1)`在整个递归过程中被调用了两次。

而我们用 decorator，在调用函数前查询一下缓存，如果没有才调用了，有了就从缓存中返回值。一下子，这个递归从二叉树式的递归成了线性的递归。

### Profiler的例子
这个例子没什么高深的，就是实用一些。

In [28]:
import cProfile, pstats, io


def profiler(func):
    def wrapper(*args, **kwargs):
        datafn = func.__name__ + ".profile"  # Name the data file
        prof = cProfile.Profile()
        retval = prof.runcall(func, *args, **kwargs)
        #prof.dump_stats(datafn)
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(prof, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return wrapper

### 注册回调函数
下面这个示例展示了通过 URL 的路由来调用相关注册的函数示例：

In [29]:
class MyApp():
    def __init__(self):
        self.func_map = {}

    def register(self, name):
        def func_wrapper(func):
            self.func_map[name] = func
            return func

        return func_wrapper

    def call_method(self, name=None):
        func = self.func_map.get(name, None)
        if func is None:
            raise Exception("No function registered against - " + str(name))
        return func()


app = MyApp()


@app.register('/')
def main_page_func():
    return "This is the main page."


@app.register('/next_page')
def next_page_func():
    return "This is the next page."


print(app.call_method('/'))
print(app.call_method('/next_page'))

This is the main page.
This is the next page.


注意：
1. 上面这个示例中，用类的实例来做decorator。
2. decorator 类中没有 `__call__()`，但是 wrapper 返回了原函数。所以，原函数没有发生任何变化。

### 给函数打日志
下面这个示例演示了一个 `logger` 的 decorator，这个 decorator 输出了函数名，参数，返回值，和运行时间。

In [31]:
from functools import wraps
from time import time

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        ts = time()
        result = fn(*args, **kwargs)
        te = time()
        print("function      = {0}".format(fn.__name__))
        print("    arguments = {0} {1}".format(args, kwargs))
        print("    return    = {0}".format(result))
        print("    time      = %.6f sec" % (te - ts))
        return result

    return wrapper


@logger
def multipy(x, y):
    return x * y


@logger
def sum_num(n):
    s = 0
    for i in range(n + 1):
        s += i
    return s


print(multipy(2, 10))
print(sum_num(100))
print(sum_num(10000000))

function      = multipy
    arguments = (2, 10) {}
    return    = 20
    time      = 0.000000 sec
20
function      = sum_num
    arguments = (100,) {}
    return    = 5050
    time      = 0.000000 sec
5050
function      = sum_num
    arguments = (10000000,) {}
    return    = 50000005000000
    time      = 0.588030 sec
50000005000000


上面那个打日志还是有点粗糙，让我们看一个更好一点的（带`log level`参数的）：

In [34]:
import inspect
from time import time


def get_line_number():
    return inspect.currentframe().f_back.f_back.f_lineno


def logger(loglevel):
    def log_decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            ts = time()
            result = fn(*args, **kwargs)
            te = time()
            print("function   = " + fn.__name__, end=' ')
            print("    arguments = {0} {1}".format(args, kwargs))
            print("    return    = {0}".format(result))
            print("    time      = %.6f sec" % (te - ts))
            if (loglevel == 'debug'):
                print("    called_from_line : " + str(get_line_number()))
            return result

        return wrapper

    return log_decorator


print(multipy(2, 10))
print(sum_num(100))
print(sum_num(10000000))

function      = multipy
    arguments = (2, 10) {}
    return    = 20
    time      = 0.000000 sec
20
function      = sum_num
    arguments = (100,) {}
    return    = 5050
    time      = 0.000000 sec
5050
function      = sum_num
    arguments = (10000000,) {}
    return    = 50000005000000
    time      = 0.591047 sec
50000005000000


但是，上面这个带log level参数的有两具不好的地方，
1. `log level`不是`debug`的时候，还是要计算函数调用的时间。
2. 不同`level`的要写在一起，不易读。

我们再接着改进：

In [36]:
import inspect
from time import time


def advance_logger(loglevel):
    def get_line_number():
        return inspect.currentframe().f_back.f_back.f_lineno

    def _basic_log(fn, result, *args, **kwargs):
        print("function   = " + fn.__name__, end=' ')
        print("    arguments = {0} {1}".format(args, kwargs))
        print("    return    = {0}".format(result))

    def info_log_decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            result = fn(*args, **kwargs)
            _basic_log(fn, result, args, kwargs)

        return wrapper

    def debug_log_decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            ts = time()
            result = fn(*args, **kwargs)
            te = time()
            _basic_log(fn, result, args, kwargs)
            print("    time      = %.6f sec" % (te - ts))
            print("    called_from_line : " + str(get_line_number()))

        return wrapper

    if loglevel is "debug":
        return debug_log_decorator
    else:
        return info_log_decorator


print(multipy(2, 10))
print(sum_num(100))
print(sum_num(10000000))

function      = multipy
    arguments = (2, 10) {}
    return    = 20
    time      = 0.000000 sec
20
function      = sum_num
    arguments = (100,) {}
    return    = 5050
    time      = 0.000000 sec
5050
function      = sum_num
    arguments = (10000000,) {}
    return    = 50000005000000
    time      = 0.582037 sec
50000005000000


你可以看到两点，
1. 我们分了两个`log level`，一个是`info`的，一个是 debug 的，然后我们在外尾根据不同的参数返回不同的 decorator。
2. 我们把`info`和`debug`中的相同的代码抽到了一个叫`_basic_log`的函数里，DRY原则。

### 线程异步
下面量个非常简单的异步执行的 decorator：

In [38]:
from threading import Thread
from functools import wraps


def async(func):
    @wraps(func)
    def async_func(*args, **kwargs):
        func_hl = Thread(target=func, args=args, kwargs=kwargs)
        func_hl.start()
        return func_hl

    return async_func


if __name__ == '__main__':
    from time import sleep

    @async
    def print_somedata():
        print('starting print_somedata')
        sleep(2)
        print('print_somedata: 2 sec passed')
        sleep(2)
        print('print_somedata: 2 sec passed')
        sleep(2)
        print('finished print_somedata')

    def main():
        print_somedata()
        print('back in main')
        print_somedata()
        print('back in main')

    main()

starting print_somedataback in main

starting print_somedataback in main



# 函数式编程的准则
**不依赖于外部的数据，而且也不改变外部数据的值，而是返回一个新的值给你。**

In [2]:
def inc(x):
    def incx(y):
        return x + y
    return incx


inc2 = inc(2)
inc5 = inc(5)

print(inc2(5))  # 输出 7
print(inc5(5))  # 输出 10

7
10


函数式编程的理念：**把函数当成变量来用，关注于描述问题而不是怎么实现**，这样可以让代码更易读

# Map & Reduce
在函数式编程中，我们不应该用循环迭代的方式，我们应该用更为高级的方法，如下所示的 Python 代码

In [3]:
name_len = list(map(len, ["hao", "chen", "coolshell"]))
print(name_len)

[3, 4, 9]


In [4]:
list(map(type, [[2, 4], 'df', 90, {'e': 5}]))

[list, str, int, dict]

**这样的代码是在描述要干什么，而不是怎么干。**

In [14]:
def toUpper(item):
    return item.upper()


upper_name = list(map(toUpper, ["hao", "chen", "coolshell"]))
print(upper_name)

['HAO', 'CHEN', 'COOLSHELL']


过程式编程，就显得不是那么清晰了：

In [15]:
upname = ['HAO', 'CHEN', 'COOLSHELL']
lowname = []
for i in range(len(upname)):
    lowname.append(upname[i].lower())

## 列表推导式

In [16]:
squares = [x * x for x in range(9)]
print(squares)

[0, 1, 4, 9, 16, 25, 36, 49, 64]


## `lambda` 表达式

下面的`lambda`表达式中有两个参数，也就是说每次从列表中取两个值，计算结果后把这个值再放回去，下面的表达式相当于：`((((1+2)+3)+4)+5)` 

In [9]:
from functools import reduce
print(reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]))

15


In [28]:
reduce(lambda x, y: x+y, ([1], [2, 3, '56g']))

[1, 2, 3, '56g']

Python 中的除了`map`和`reduce`外，还有一些别的如`filter`, `find`, `all`, `any`的函数做辅助（其它函数式的语言也有），可以让你的代码更简洁，更易读。 

In [39]:
def is_float(x):
    return isinstance(x, float)
    
list(filter(is_float, [1, 3, 4., '4', 3.5]))

[4.0, 3.5]

In [40]:
for i in filter(is_float, [1, 3, 4., '4', 3.5]):
    print(i)

4.0
3.5


In [52]:
print(all([1, 2, 3]))
print(all(''))
print(all([]))
print(all(['']))
print(all([1, 0]))
print(all([1, 3, []]))
print(all(range(4)))

True
True
True
False
False
False
False


In [56]:
any([1, 3, 0])

True

## 更复杂的例子
### 计算数组中正数的平均值

先用过程式编程

In [18]:
num = [2, -5, 9, 7, -2, 5, 3, 1, 0, -3, 8]
positive_num_cnt = 0
positive_num_sum = 0
for i in range(len(num)):
    if num[i] > 0:
        positive_num_cnt += 1
        positive_num_sum += num[i]

if positive_num_cnt > 0:
    average = positive_num_sum / positive_num_cnt

print(average)

5.0


如果用函数式编程，这个例子可以写成这样：

In [58]:
from functools import reduce
num = [2, -5, 9, 7, -2, 5, 3, 1, 0, -3, 8]
positive_num = [x for x in num if x > 0]
average = reduce(lambda x, y: x+y, positive_num) / len(positive_num)

前面提到过多次的函数式编程关注的是：describe what to do, rather than how to do it. 于是，我们把以前的过程式的编程范式叫做 [Imperative Programming](https://en.wikipedia.org/wiki/Imperative_programming) – 指令式编程，而把函数式的这种范式叫做 [Declarative Programming](https://en.wikipedia.org/wiki/Declarative_programming) – 声明式编程。

### 情境：给定$3$辆车有$70\%$的概率可以往前走一步，一共有$5$次机会，我们打出每一次这$3$辆车的前行状态：

In [60]:
from random import random

time = 5
car_positions = [1, 1, 1]

while time:
    # decrease time
    time -= 1

    print(10*'*')
    for i in range(len(car_positions)):
        # move car
        if random() > 0.3:
            car_positions[i] += 1

        # draw car
        print('-' * car_positions[i])

**********
--
--
--
**********
---
--
--
**********
----
---
---
**********
-----
----
---
**********
------
----
----


我们可以把这个两重循环变成一些函数模块，这样有利于我们更容易地阅读代码：

In [21]:
from random import random


def move_cars():
    for i, _ in enumerate(car_positions):
        if random() > 0.3:
            car_positions[i] += 1


def draw_car(car_position):
    print('-' * car_position)


def run_step_of_race():
    global time
    time -= 1
    move_cars()


def draw():
    print('')
    for car_position in car_positions:
        draw_car(car_position)


time = 5
car_positions = [1, 1, 1]

while time:
    run_step_of_race()
    draw()


-
-
-

--
--
--

---
--
---

----
--
----

-----
--
----


上面的代码，我们可以从主循环开始，我们可以很清楚地看到程序的主干，因为我们把程序的逻辑分成了几个函数，这样一来，我们的代码逻辑也会变得几个小碎片，于是我们读代码时要考虑的上下文就少了很多，阅读代码也会更容易。不像第一个示例，如果没有注释和说明，你还是需要花些时间理解一下。而**把代码逻辑封装成了函数后，我们就相当于给每个相对独立的程序逻辑取了个名字，于是代码成了自解释的。**

但是，你会发现，封装成函数后，这些函数都会依赖于共享的变量来同步其状态。于是，我们在读代码的过程时，每当我们进入到函数里，一量读到访问了一个外部的变量，我们马上要去查看这个变量的上下文，然后还要在大脑里推演这个变量的状态， 我们才知道程序的真正逻辑。也就是说，**这些函数间必需知道其它函数是怎么修改它们之间的共享变量的，所以，这些函数是有状态的。**

我们知道，有状态并不是一件很好的事情，无论是对代码重用，还是对代码的并行来说，都是有副作用的。因此，我们要想个方法把这些状态搞掉，于是出现了我们的 `Functional Programming` 的编程范式。下面，我们来看看函数式的方式应该怎么写？

In [22]:
from random import random


def move_cars(car_positions):
    return [x + 1 if random() > 0.3 else x for x in car_positions]


def output_car(car_position):
    return '-' * car_position


def run_step_of_race(state):
    return {'time': state['time'] - 1,
            'car_positions': move_cars(state['car_positions'])}


def draw(state):
    print('')
    print('\n'.join(map(output_car, state['car_positions'])))


def race(state):
    draw(state)
    if state['time']:
        race(run_step_of_race(state))


race({'time': 5,
      'car_positions': [1, 1, 1]})


-
-
-

-
--
--

--
--
--

---
---
--

---
----
--

----
-----
---


上面的代码依然把程序的逻辑分成了函数，不过这些函数都是`functional`的。因为它们有三个症状：
1. 它们之间没有共享的变量。
2. 函数间通过参数和返回值来传递数据。
3. 在函数里没有临时变量。

我们还可以看到，`for`循环被递归取代了（见`race`函数）—— 递归是函数式编程中带用到的技术，正如前面所说的，**递归的本质就是描述问题是什么。**

# `Pipeline`
`pipeline` 管道借鉴于`Unix Shell`的管道操作——把若干个命令串起来，前面命令的输出成为后面命令的输入，如此完成一个流式计算。（注：管道绝对是一个伟大的发明，他的设哲学就是**KISS** – 让每个功能就做一件事，并把这件事做到极致，软件或程序的拼装会变得更为简单和直观。这个设计理念影响非常深远，包括今天的 Web Service，云计算，以及大数据的流式计算等等）

比如，我们如下的`shell`命令：

```sh
ps auwwx | awk '{print $2}' | sort -n | xargs echo
```

如果我们抽象成函数式的语言，就像下面这样：

```py
xargs(echo, sort(n, awk('print $2', ps(auwwx))))
```

In [62]:
class Pipe(object):
    def __init__(self, func):
        self.func = func

    def __ror__(self, other):
        def generator():
            for obj in other:
                if obj is not None:
                    yield self.func(obj)
        return generator()


@Pipe
def even_filter(num):
    return num if num % 2 == 0 else None


@Pipe
def multiply_by_three(num):
    return num*3


@Pipe
def convert_to_string(num):
    return 'The Number: %s' % num


@Pipe
def echo(item):
    print(item)
    return item


def force(sqs):
    for item in sqs:
        pass


nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

force(nums | even_filter | multiply_by_three | convert_to_string | echo)

The Number: 6
The Number: 12
The Number: 18
The Number: 24
The Number: 30


In [82]:
@Pipe
def g(x):
    y = x * 2
    print(y)

In [83]:
force([2, 5, 7, 8]|g)

4
10
14
16
